In [1]:
!pip install fastapi uvicorn opencv-python numpy pillow python-multipart pyngrok --quiet


In [2]:
from pyngrok import conf
conf.get_default().auth_token = "30hV47iXDAVoAmROyCHRGAhz5KK_21EVeup1ekovrXV7URmNm"


In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse
import cv2
import numpy as np
import base64
from io import BytesIO
from PIL import Image
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import asyncio

app = FastAPI()
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

@app.get("/")
def root():
    return {"message": "Face Extractor API is running."}

@app.post("/extract/base64")
async def extract_face_base64(file: UploadFile = File(...)):
    contents = await file.read()
    img_np = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)

    if img is None:
        raise HTTPException(status_code=400, detail="Invalid image")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        raise HTTPException(status_code=422, detail="No face found")

    x, y, w, h = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
    face = img[y:y+h, x:x+w]

    _, buffer = cv2.imencode('.jpg', face)
    face_b64 = base64.b64encode(buffer).decode("utf-8")

    return {"base64_image": face_b64}

@app.post("/extract/image")
async def extract_face_image(file: UploadFile = File(...)):
    contents = await file.read()
    img_np = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)

    if img is None:
        raise HTTPException(status_code=400, detail="Invalid image")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        raise HTTPException(status_code=422, detail="No face found")

    x, y, w, h = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
    face = img[y:y+h, x:x+w]
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(face_rgb)

    buffer = BytesIO()
    pil_img.save(buffer, format="PNG")
    buffer.seek(0)

    return StreamingResponse(buffer, media_type="image/png")


# Start FastAPI via Uvicorn and expose with ngrok
def run_app():
    nest_asyncio.apply()
    public_url = ngrok.connect(8000)
    print(f"🔗 Public API URL: {public_url}/docs")
    uvicorn.run(app, port=8000)

run_app()


INFO:     Started server process [999]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🔗 Public API URL: NgrokTunnel: "https://7c48a516a154.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     2001:8f8:1269:6e7d:540f:c20f:f087:100e:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:8f8:1269:6e7d:540f:c20f:f087:100e:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2001:8f8:1d08:4855:3c27:163c:9b68:d213:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:8f8:1269:6e7d:9c5a:df67:73a8:4326:0 - "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 Not Found
INFO:     2001:8f8:1269:6e7d:9c5a:df67:73a8:4326:0 - "GET /apple-touch-icon.png HTTP/1.1" 404 Not Found
INFO:     2001:8f8:1269:6e7d:9c5a:df67:73a8:4326:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2001:8f8:1269:6e7d:9c5a:df67:73a8:4326:0 - "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 Not Found
INFO:     2001:8f8:1269:6e7d:9c5a:df67:73a8:4326:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2001:8f8:1269:6e7d:9c5a:df67:73a8:4326:0 - "GET /apple-touch-icon.png HTTP/1.1" 404 Not Found
INFO:     2001:8f8:1269:6e7d:1df3:3a1d:b6df:4428:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:8f8:1269:6e7d:1df3:3a1d